## Install Dependencies

### Modules

In [ ]:
!pip install flask
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
!pip install pandas
!pip install yfinance
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask

--2020-12-13 01:24:01--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 216.105.38.13
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|216.105.38.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2020-12-13 01:24:01--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://versaweb.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2020-12-13 01:24:01--  https://versaweb.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving versaweb.dl.so

### Dependencies

In [ ]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

import numpy as np
import random
from collections import deque


### Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/My Drive/Reinforcement Learning/RL Project/candlestick-screener-master/candlestick-screener-master/datasets/daily

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### TPU Initialization

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

### Patterns

In [ ]:
candlestick_patterns = {
    'CDL2CROWS':'Two Crows',
    'CDL3BLACKCROWS':'Three Black Crows',
    'CDL3INSIDE':'Three Inside Up/Down',
    'CDL3LINESTRIKE':'Three-Line Strike',
    'CDL3OUTSIDE':'Three Outside Up/Down',
    'CDL3STARSINSOUTH':'Three Stars In The South',
    'CDL3WHITESOLDIERS':'Three Advancing White Soldiers',
    'CDLABANDONEDBABY':'Abandoned Baby',
    'CDLADVANCEBLOCK':'Advance Block',
    'CDLBELTHOLD':'Belt-hold',
    'CDLBREAKAWAY':'Breakaway',
    'CDLCLOSINGMARUBOZU':'Closing Marubozu',
    'CDLCONCEALBABYSWALL':'Concealing Baby Swallow',
    'CDLCOUNTERATTACK':'Counterattack',
    'CDLDARKCLOUDCOVER':'Dark Cloud Cover',
    'CDLDOJI':'Doji',
    'CDLDOJISTAR':'Doji Star',
    'CDLDRAGONFLYDOJI':'Dragonfly Doji',
    'CDLENGULFING':'Engulfing Pattern',
    'CDLEVENINGDOJISTAR':'Evening Doji Star',
    'CDLEVENINGSTAR':'Evening Star',
    'CDLGAPSIDESIDEWHITE':'Up/Down-gap side-by-side white lines',
    'CDLGRAVESTONEDOJI':'Gravestone Doji',
    'CDLHAMMER':'Hammer',
    'CDLHANGINGMAN':'Hanging Man',
    'CDLHARAMI':'Harami Pattern',
    'CDLHARAMICROSS':'Harami Cross Pattern',
    'CDLHIGHWAVE':'High-Wave Candle',
    'CDLHIKKAKE':'Hikkake Pattern',
    'CDLHIKKAKEMOD':'Modified Hikkake Pattern',
    'CDLHOMINGPIGEON':'Homing Pigeon',
    'CDLIDENTICAL3CROWS':'Identical Three Crows',
    'CDLINNECK':'In-Neck Pattern',
    'CDLINVERTEDHAMMER':'Inverted Hammer',
    'CDLKICKING':'Kicking',
    'CDLKICKINGBYLENGTH':'Kicking - bull/bear determined by the longer marubozu',
    'CDLLADDERBOTTOM':'Ladder Bottom',
    'CDLLONGLEGGEDDOJI':'Long Legged Doji',
    'CDLLONGLINE':'Long Line Candle',
    'CDLMARUBOZU':'Marubozu',
    'CDLMATCHINGLOW':'Matching Low',
    'CDLMATHOLD':'Mat Hold',
    'CDLMORNINGDOJISTAR':'Morning Doji Star',
    'CDLMORNINGSTAR':'Morning Star',
    'CDLONNECK':'On-Neck Pattern',
    'CDLPIERCING':'Piercing Pattern',
    'CDLRICKSHAWMAN':'Rickshaw Man',
    'CDLRISEFALL3METHODS':'Rising/Falling Three Methods',
    'CDLSEPARATINGLINES':'Separating Lines',
    'CDLSHOOTINGSTAR':'Shooting Star',
    'CDLSHORTLINE':'Short Line Candle',
    'CDLSPINNINGTOP':'Spinning Top',
    'CDLSTALLEDPATTERN':'Stalled Pattern',
    'CDLSTICKSANDWICH':'Stick Sandwich',
    'CDLTAKURI':'Takuri (Dragonfly Doji with very long lower shadow)',
    'CDLTASUKIGAP':'Tasuki Gap',
    'CDLTHRUSTING':'Thrusting Pattern',
    'CDLTRISTAR':'Tristar Pattern',
    'CDLUNIQUE3RIVER':'Unique 3 River',
    'CDLUPSIDEGAP2CROWS':'Upside Gap Two Crows',
    'CDLXSIDEGAP3METHODS':'Upside/Downside Gap Three Methods'
}

## Prediction

### Agent

In [ ]:
class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.90
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = load_model("/content/drive/My Drive/Reinforcement Learning/RL Project/models/" + model_name) if is_eval else self._model()

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))

        return model

    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])
    
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

### Environment

In [ ]:
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import itertools


class TradingEnv(gym.Env):

  def __init__(self, train_data, init_invest=20000):
    # data
    self.stock_price_history = np.around(train_data) # round up to integer to reduce state space
    self.n_stock, self.n_step = self.stock_price_history.shape

    # instance attributes
    self.init_invest = init_invest
    self.cur_step = None
    self.stock_owned = None
    self.stock_price = None
    self.cash_in_hand = None

    # action space
    self.action_space = spaces.Discrete(3**self.n_stock)

    # observation space: give estimates in order to sample and build scaler
    stock_max_price = self.stock_price_history.max(axis=1)
    stock_range = [[0, init_invest * 2 // mx] for mx in stock_max_price]
    price_range = [[0, mx] for mx in stock_max_price]
    cash_in_hand_range = [[0, init_invest * 2]]
    self.observation_space = spaces.MultiDiscrete(stock_range + price_range + cash_in_hand_range)

    # seed and start
    self._seed()
    self._reset()


  def _seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]


  def _reset(self):
    self.cur_step = 0
    self.stock_owned = [0] * self.n_stock
    self.stock_price = self.stock_price_history[:, self.cur_step]
    self.cash_in_hand = self.init_invest
    return self._get_obs()


  def _step(self, action):
    assert self.action_space.contains(action)
    prev_val = self._get_val()
    self.cur_step += 1
    self.stock_price = self.stock_price_history[:, self.cur_step] # update price
    self._trade(action)
    cur_val = self._get_val()
    reward = cur_val - prev_val
    done = self.cur_step == self.n_step - 1
    info = {'cur_val': cur_val}
    return self._get_obs(), reward, done, info


  def _get_obs(self):
    obs = []
    obs.extend(self.stock_owned)
    obs.extend(list(self.stock_price))
    obs.append(self.cash_in_hand)
    return obs


  def _get_val(self):
    return np.sum(self.stock_owned * self.stock_price) + self.cash_in_hand


  def _trade(self, action):
    # all combo to sell(0), hold(1), or buy(2) stocks
    action_combo = map(list, itertools.product([0, 1, 2], repeat=self.n_stock))
    action_vec = action_combo[action]

    # one pass to get sell/buy index
    sell_index = []
    buy_index = []
    for i, a in enumerate(action_vec):
      if a == 0:
        sell_index.append(i)
      elif a == 2:
        buy_index.append(i)

    # two passes: sell first, then buy; might be naive in real-world settings
    if sell_index:
      for i in sell_index:
        self.cash_in_hand += self.stock_price[i] * self.stock_owned[i]
        self.stock_owned[i] = 0
    if buy_index:
      can_buy = True
      while can_buy:
        for i in buy_index:
          if self.cash_in_hand > self.stock_price[i]:
            self.stock_owned[i] += 1 # buy one share
            self.cash_in_hand -= self.stock_price[i]
          else:
            can_buy = False

### State Functions

In [ ]:
import math

In [ ]:
def formatPrice(n):
    return("-$" if n<0 else "$")+"{0:.2f}".format(abs(n))

def getStockDataVec(key):
    vec = []
    lines = open("/content/drive/My Drive/Reinforcement Learning/RL Project/datasets/daily/"+key+".csv","r").read().splitlines()
    
    for line in lines[1:]:
        vec.append(float(line.split(",")[4]))
    return vec 

def sigmoid(x):
    return 1/(1+math.exp(-x))

def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))

    return np.array([res])

### Train Model

In [ ]:
def training(s_name,w_size=10,e_count=1):
  import sys

  """if len(sys.argv) != 4:
      print("Usage: python train.py [stock] [window] [episodes]")
      exit()
  """
  #stock_name, window_size, episode_count = sys.argv[1], int(sys.argv[2]), int(sys.argv[3])
  stock_name = s_name#input("Enter stock_name, window_size, Episode_count")
  window_size = w_size#input()
  episode_count = e_count#input()
  stock_name = str(stock_name)
  window_size = int(window_size)
  episode_count = int(episode_count)


  agent = Agent(window_size)
  data = getStockDataVec(stock_name)
  l = len(data) - 1
  batch_size = 32

  for e in range(episode_count + 1):
      print("Episode " + str(e) + "/" + str(episode_count))
      state = getState(data, 0, window_size + 1)

      total_profit = 0
      agent.inventory = []

      for t in range(l):
          action = agent.act(state)

          # sit
          next_state = getState(data, t + 1, window_size + 1)
          reward = 0

          if action == 1: # buy
              agent.inventory.append(data[t])
              print("Buy: " + formatPrice(data[t]))

          elif action == 2 and len(agent.inventory) > 0: # sell
              bought_price = agent.inventory.pop(0)
              reward = max(data[t] - bought_price, 0)
              total_profit += data[t] - bought_price
              print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

          done = True if t == l - 1 else False
          agent.memory.append((state, action, reward, next_state, done))
          state = next_state

          if done:
              print("--------------------------------")
              print("Total Profit: " + formatPrice(total_profit))
              print("--------------------------------")
          if len(agent.memory) > batch_size:
              agent.expReplay(batch_size)

  agent.model.save("/content/drive/My Drive/Reinforcement Learning/RL Project/models/model_ep" + str(e))
  return formatPrice(total_profit)

In [ ]:
training('AAPL',3,1)

Episode 0/1
Buy: $118.69
Sell: $116.32 | Profit: -$2.37
Buy: $119.21
Buy: $119.39
Sell: $113.85 | Profit: -$5.36
Buy: $116.03
Buy: $116.59
Buy: $123.08
Sell: $122.94 | Profit: $3.55
Buy: $122.25
--------------------------------
Total Profit: -$4.18
--------------------------------
Episode 1/1
Buy: $110.44
Buy: $119.03
Buy: $116.32
Sell: $115.97 | Profit: $5.53
Buy: $119.49
Sell: $119.26 | Profit: $0.23
Sell: $120.30 | Profit: $3.98
Sell: $119.39 | Profit: -$0.10
Buy: $119.05
Sell: $122.72 | Profit: $3.67
Buy: $122.25
--------------------------------
Total Profit: $13.31
--------------------------------
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Reinforcement Learning/RL Project/models/model_ep1/assets


'$13.31'

### Test Model

In [ ]:
#evaluating models 

"""if len(sys.argv) != 3:
    print("Usage: python evaluate.py [stock] [model]")
    exit()
"""

#stock_name, model_name = sys.argv[1], sys.argv[2] input("Enter Stock_name, Model_name")
stock_name = "AAPL"
model_name = "model_ep1"


model = load_model("/content/drive/My Drive/Reinforcement Learning/RL Project/models/" + model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = []

for t in range(l):
    action = agent.act(state)

    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0

    if action == 1: # buy
        agent.inventory.append(data[t])
        print("Buy: " + formatPrice(data[t]))

    elif action == 2 and len(agent.inventory) > 0: # sell
        bought_price = agent.inventory.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += data[t] - bought_price
        print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state

    if done:
        print("--------------------------------")
        print(stock_name + " Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")
        print ("Total profit is:",formatPrice(total_profit))

--------------------------------
AAPL Total Profit: $0.00
--------------------------------
Total profit is: $0.00


In [ ]:
#print(len(data))

In [ ]:
import os
stock_symbols=[]
for filename in os.listdir('/content/drive/My Drive/Reinforcement Learning/RL Project/datasets/daily'):
  stock_symbols.append(filename)

In [ ]:
stock_symbols

['MMM.csv',
 'ABMD.csv',
 'ABT.csv',
 'ABBV.csv',
 'ACN.csv',
 'ADBE.csv',
 'ATVI.csv',
 'AES.csv',
 'A.csv',
 'AAP.csv',
 'AFL.csv',
 'AMD.csv',
 'APD.csv',
 'AKAM.csv',
 'ALK.csv',
 'ALB.csv',
 'ARE.csv',
 'ALGN.csv',
 'AGN.csv',
 'ALXN.csv',
 'ALLE.csv',
 'LNT.csv',
 'ALL.csv',
 'ADS.csv',
 'GOOGL.csv',
 'GOOG.csv',
 'MO.csv',
 'AMZN.csv',
 'AMCR.csv',
 'AEE.csv',
 'AAL.csv',
 'AXP.csv',
 'AIG.csv',
 'AMT.csv',
 'AEP.csv',
 'AWK.csv',
 'ABC.csv',
 'AMP.csv',
 'AME.csv',
 'AMGN.csv',
 'ADI.csv',
 'APH.csv',
 'AON.csv',
 'ANTM.csv',
 'ANSS.csv',
 'APA.csv',
 'AOS.csv',
 'AIV.csv',
 'AAPL.csv',
 'AMAT.csv',
 'APTV.csv',
 'ADM.csv',
 'ANET.csv',
 'ARNC.csv',
 'AJG.csv',
 'AIZ.csv',
 'ATO.csv',
 'T.csv',
 'ADSK.csv',
 'ADP.csv',
 'AZO.csv',
 'AVB.csv',
 'BKR.csv',
 'AVY.csv',
 'BLL.csv',
 'BAC.csv',
 'BK.csv',
 'BAX.csv',
 'BRK.B.csv',
 'BBY.csv',
 'BDX.csv',
 'BIIB.csv',
 'BLK.csv',
 'BA.csv',
 'BKNG.csv',
 'BXP.csv',
 'BWA.csv',
 'BSX.csv',
 'AVGO.csv',
 'BMY.csv',
 'BR.csv',
 'BF.B.cs

## App

In [ ]:
import os, csv
import talib
import yfinance as yf
import pandas
from flask import Flask, escape, request, render_template

app = Flask(__name__, template_folder='/content/drive/My Drive/Reinforcement Learning/RL Project/templates')
run_with_ngrok(app)

@app.route('/update_data')
def snapshot():
    with open('/content/drive/My Drive/Reinforcement Learning/RL Project/datasets/symbols.csv') as f:
        for line in f:
            if "," not in line:
                continue
            symbol = line.split(",")[0]
            data = yf.download(symbol, start="2020-11-01", end="2020-12-11")
            data.to_csv('/content/drive/My Drive/Reinforcement Learning/RL Project/datasets/daily/{}.csv'.format(symbol))

    return {
        "code": "success"
    }

@app.route('/')
def index():
    pattern  = request.args.get('pattern', False)
    stocks = {}

    with open('/content/drive/My Drive/Reinforcement Learning/RL Project/datasets/symbols.csv') as f:
        for row in csv.reader(f):
            stocks[row[0]] = {'company': row[1]}

    if pattern:
        for filename in os.listdir('/content/drive/My Drive/Reinforcement Learning/RL Project/datasets/daily'):
            df = pandas.read_csv('/content/drive/My Drive/Reinforcement Learning/RL Project/datasets/daily/{}'.format(filename))
            pattern_function = getattr(talib, pattern)
            symbol = filename.split('.')[0]

            try:
                results = pattern_function(df['Open'], df['High'], df['Low'], df['Close'])
                last = results.tail(1).values[0]

                if last > 0:
                    stocks[symbol][pattern] = 'bullish'
                elif last < 0:
                    stocks[symbol][pattern] = 'bearish'
                else:
                    stocks[symbol][pattern] = None
            except Exception as e:
                print('failed on filename: ', filename)

    return render_template('index.html', candlestick_patterns=candlestick_patterns, stocks=stocks, pattern=pattern)

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9e1db204d2b2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Dec/2020 01:26:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2020 01:26:36] "GET /favicon.ico HTTP/1.1" 404 -


failed on filename:  AGN.csv
failed on filename:  BRK.B.csv
failed on filename:  BF.B.csv
failed on filename:  ETFC.csv
failed on filename:  RTN.csv
failed on filename:  UTX.csv
failed on filename:  WCG.csv


127.0.0.1 - - [13/Dec/2020 01:28:38] "GET /?pattern=CDLDOJISTAR HTTP/1.1" 200 -


References

1) [Deep Reinforcement Learning for Automated Stock Trading Ensemble Strategy ICAIF 2020](https://github.com/AI4Finance-LLC/Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020)

2) [Q Learning for Trading](https://github.com/llSourcell/Q-Learning-for-Trading)

3) [Candle Stick Screener](https://github.com/hackingthemarkets/candlestick-screener)

4) [Part Time Larry, 9 Aug 2020, Finding Breakout Candidates with Python and Pandas](https://www.youtube.com/watch?v=exGuyBnhN_8)